In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performerErrorTypeTest_V1 as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [3]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return replaceTAGS(tokens)

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [10]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce-binary-accuracy"
        Input_Path = "Trianing\InputTxt\Split-500-reduce-binary-accuracy"
        model_for_training_org_path = "Model-for-training-org\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\weight-1vs1-v1"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        #get all txt file in input path
        target_max_len = 0
        token_num = 0

        #start training
        from Data_buffer import DataGeneratorTrainErrorTypeEdition as DGTrain
        from Data_buffer import DataGeneratorValidationErrorTypeEdition as DGValidation
        from Data_buffer import DataBuffer as db
        from random import randrange
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #load source_max_len
        source_max_len = loadDictionary(Trained_model_Path + "/" + source_max_len_name)
        #load training models and their len
        x_train = loadTestTrainData(model_for_training_path + "/" + "x_train_0.npy")
        x_validation = loadTestTrainData(model_for_training_path + "/" + "x_validation_0.npy")
        
        buffer_train_num = len(x_train)
        buffer_val_train_num = len(x_validation)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              errNum=36,
                              token_num=len(source_token_dict),
                              embed_dim=32, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        #''' <-----traing switch
        #Start training
        
        print("Strat training...")
        
        #reload model for training 
        x_model_name = "x_train_0.npy"
        y0_model_name = "y_train[0]_0.npy"
        x_model = loadTestTrainData(model_for_training_path + "/" + x_model_name)
        y0_model = loadTestTrainData(model_for_training_path + "/" + y0_model_name)
        
        #set check point 
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit(
                            x = x_model,
                            y = [y0_model],
                            epochs = 1000, #100 200 500 3000
                            verbose = 2, #set visibility
                            validation_data = validation_generator,
                            callbacks = [model_checkpoint_callback],
                            )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [11]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [12]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [13]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [14]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [15]:
x=TestTranslate()
x.test_translate()

Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Creating training generator...
Creating validation generator...
Strat training...


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
22/22 - 25s - loss: 0.2747 - binary_accuracy: 0.9202 - val_loss: 0.2067 - val_binary_accuracy: 0.9537
Epoch 2/1000


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


22/22 - 18s - loss: 0.2009 - binary_accuracy: 0.9539 - val_loss: 0.1964 - val_binary_accuracy: 0.9540
Epoch 3/1000
22/22 - 18s - loss: 0.1940 - binary_accuracy: 0.9539 - val_loss: 0.1937 - val_binary_accuracy: 0.9538
Epoch 4/1000
22/22 - 18s - loss: 0.1899 - binary_accuracy: 0.9538 - val_loss: 0.1914 - val_binary_accuracy: 0.9535
Epoch 5/1000
22/22 - 18s - loss: 0.1865 - binary_accuracy: 0.9541 - val_loss: 0.1879 - val_binary_accuracy: 0.9536
Epoch 6/1000
22/22 - 18s - loss: 0.1848 - binary_accuracy: 0.9542 - val_loss: 0.1862 - val_binary_accuracy: 0.9536
Epoch 7/1000
22/22 - 18s - loss: 0.1851 - binary_accuracy: 0.9537 - val_loss: 0.1860 - val_binary_accuracy: 0.9539
Epoch 8/1000
22/22 - 18s - loss: 0.1840 - binary_accuracy: 0.9540 - val_loss: 0.1854 - val_binary_accuracy: 0.9539
Epoch 9/1000
22/22 - 18s - loss: 0.1850 - binary_accuracy: 0.9540 - val_loss: 0.1845 - val_binary_accuracy: 0.9539
Epoch 10/1000
22/22 - 17s - loss: 0.1839 - binary_accuracy: 0.9540 - val_loss: 0.1846 - val_b

...

Epoch 700/1000
22/22 - 12s - loss: 5.3298e-04 - binary_accuracy: 0.9998 - val_loss: 0.4528 - val_binary_accuracy: 0.9554
Epoch 701/1000
22/22 - 17s - loss: 3.8471e-04 - binary_accuracy: 0.9999 - val_loss: 0.4465 - val_binary_accuracy: 0.9559
Epoch 702/1000
22/22 - 17s - loss: 5.3405e-04 - binary_accuracy: 0.9999 - val_loss: 0.4504 - val_binary_accuracy: 0.9560
Epoch 703/1000
22/22 - 12s - loss: 4.5999e-04 - binary_accuracy: 0.9999 - val_loss: 0.4448 - val_binary_accuracy: 0.9543
Epoch 704/1000
22/22 - 17s - loss: 3.7136e-04 - binary_accuracy: 0.9999 - val_loss: 0.4462 - val_binary_accuracy: 0.9561
Epoch 705/1000
22/22 - 12s - loss: 4.1030e-04 - binary_accuracy: 0.9999 - val_loss: 0.4425 - val_binary_accuracy: 0.9562
Epoch 706/1000
22/22 - 17s - loss: 3.1482e-04 - binary_accuracy: 1.0000 - val_loss: 0.4492 - val_binary_accuracy: 0.9569
Epoch 707/1000
22/22 - 17s - loss: 3.0855e-04 - binary_accuracy: 0.9999 - val_loss: 0.4448 - val_binary_accuracy: 0.9561
Epoch 708/1000
22/22 - 12s - los

Epoch 768/1000
22/22 - 17s - loss: 2.5295e-04 - binary_accuracy: 0.9999 - val_loss: 0.4738 - val_binary_accuracy: 0.9564
Epoch 769/1000
22/22 - 17s - loss: 2.2227e-04 - binary_accuracy: 0.9999 - val_loss: 0.4668 - val_binary_accuracy: 0.9557
Epoch 770/1000
22/22 - 12s - loss: 2.1467e-04 - binary_accuracy: 0.9999 - val_loss: 0.4759 - val_binary_accuracy: 0.9560
Epoch 771/1000
22/22 - 17s - loss: 2.7319e-04 - binary_accuracy: 0.9999 - val_loss: 0.4725 - val_binary_accuracy: 0.9555
Epoch 772/1000
22/22 - 12s - loss: 3.7443e-04 - binary_accuracy: 0.9999 - val_loss: 0.4763 - val_binary_accuracy: 0.9562
Epoch 773/1000
22/22 - 17s - loss: 1.9968e-04 - binary_accuracy: 0.9999 - val_loss: 0.4743 - val_binary_accuracy: 0.9557
Epoch 774/1000
22/22 - 17s - loss: 2.4863e-04 - binary_accuracy: 0.9999 - val_loss: 0.4780 - val_binary_accuracy: 0.9556
Epoch 775/1000
22/22 - 12s - loss: 2.0663e-04 - binary_accuracy: 0.9999 - val_loss: 0.4810 - val_binary_accuracy: 0.9555
Epoch 776/1000
22/22 - 17s - los

Epoch 836/1000
22/22 - 12s - loss: 2.5095e-04 - binary_accuracy: 0.9999 - val_loss: 0.4970 - val_binary_accuracy: 0.9557
Epoch 837/1000
22/22 - 17s - loss: 2.4075e-04 - binary_accuracy: 0.9999 - val_loss: 0.4949 - val_binary_accuracy: 0.9558
Epoch 838/1000
22/22 - 12s - loss: 1.9853e-04 - binary_accuracy: 0.9999 - val_loss: 0.4911 - val_binary_accuracy: 0.9557
Epoch 839/1000
22/22 - 17s - loss: 2.0988e-04 - binary_accuracy: 0.9999 - val_loss: 0.4971 - val_binary_accuracy: 0.9557
Epoch 840/1000
22/22 - 17s - loss: 2.1215e-04 - binary_accuracy: 0.9999 - val_loss: 0.4931 - val_binary_accuracy: 0.9557
Epoch 841/1000
22/22 - 12s - loss: 2.5414e-04 - binary_accuracy: 0.9999 - val_loss: 0.4945 - val_binary_accuracy: 0.9561
Epoch 842/1000
22/22 - 17s - loss: 2.8922e-04 - binary_accuracy: 0.9999 - val_loss: 0.4915 - val_binary_accuracy: 0.9553
Epoch 843/1000
22/22 - 12s - loss: 4.7551e-04 - binary_accuracy: 0.9999 - val_loss: 0.4871 - val_binary_accuracy: 0.9559
Epoch 844/1000
22/22 - 12s - los

Epoch 904/1000
22/22 - 17s - loss: 2.6044e-04 - binary_accuracy: 0.9999 - val_loss: 0.5131 - val_binary_accuracy: 0.9558
Epoch 905/1000
22/22 - 12s - loss: 1.8866e-04 - binary_accuracy: 0.9999 - val_loss: 0.5104 - val_binary_accuracy: 0.9563
Epoch 906/1000
22/22 - 17s - loss: 1.7276e-04 - binary_accuracy: 0.9999 - val_loss: 0.5061 - val_binary_accuracy: 0.9555
Epoch 907/1000
22/22 - 12s - loss: 1.9127e-04 - binary_accuracy: 0.9999 - val_loss: 0.5132 - val_binary_accuracy: 0.9561
Epoch 908/1000
22/22 - 17s - loss: 2.3458e-04 - binary_accuracy: 0.9999 - val_loss: 0.5068 - val_binary_accuracy: 0.9561
Epoch 909/1000
22/22 - 17s - loss: 1.8933e-04 - binary_accuracy: 0.9999 - val_loss: 0.5137 - val_binary_accuracy: 0.9555
Epoch 910/1000
22/22 - 12s - loss: 2.2013e-04 - binary_accuracy: 0.9999 - val_loss: 0.5054 - val_binary_accuracy: 0.9555
Epoch 911/1000
22/22 - 17s - loss: 2.6435e-04 - binary_accuracy: 0.9999 - val_loss: 0.5119 - val_binary_accuracy: 0.9555
Epoch 912/1000
22/22 - 12s - los

Epoch 972/1000
22/22 - 12s - loss: 2.3498e-04 - binary_accuracy: 0.9999 - val_loss: 0.5272 - val_binary_accuracy: 0.9555
Epoch 973/1000
22/22 - 17s - loss: 2.7687e-04 - binary_accuracy: 0.9999 - val_loss: 0.5275 - val_binary_accuracy: 0.9554
Epoch 974/1000
22/22 - 17s - loss: 2.2354e-04 - binary_accuracy: 0.9999 - val_loss: 0.5239 - val_binary_accuracy: 0.9552
Epoch 975/1000
22/22 - 17s - loss: 1.9522e-04 - binary_accuracy: 0.9999 - val_loss: 0.5240 - val_binary_accuracy: 0.9552
Epoch 976/1000
22/22 - 12s - loss: 1.7166e-04 - binary_accuracy: 0.9999 - val_loss: 0.5289 - val_binary_accuracy: 0.9554
Epoch 977/1000
22/22 - 17s - loss: 2.5588e-04 - binary_accuracy: 0.9999 - val_loss: 0.5268 - val_binary_accuracy: 0.9553
Epoch 978/1000
22/22 - 17s - loss: 2.5852e-04 - binary_accuracy: 0.9999 - val_loss: 0.5310 - val_binary_accuracy: 0.9555
Epoch 979/1000
22/22 - 12s - loss: 1.9649e-04 - binary_accuracy: 0.9999 - val_loss: 0.5255 - val_binary_accuracy: 0.9553
Epoch 980/1000
22/22 - 3s - loss